# Testing Model Predictions

## VERSIONS

- 01: 
    - Initial file


## Imports

In [1]:
import os
import io
import sys
from pathlib import Path
from importlib.metadata import version
from importlib.metadata import PackageNotFoundError  # For handling package version errors
from logging import Logger  # For type hinting
from typing import List, Optional  # For type hinting
import json
import logging
import argparse
import time
import joblib
import pandas as pd
from pandas.errors import ParserError
import torch
from torch.nn import Module  # For type hinting
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

# from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

In [2]:
package_list = ["pandas", "seaborn", "matplotlib", "torch", "joblib", "tqdm"]
for package in package_list:
    try:
        print(f"{package} version: {version(package)}")  # Raises PackageNotFoundError if not found
    except PackageNotFoundError:
        print(f"❌ Package '{package}' not found. Please install it.")

pandas version: 2.3.2
seaborn version: 0.13.2
matplotlib version: 3.10.6
torch version: 2.5.1
joblib version: 1.5.2
tqdm version: 4.67.1


## Global Variables

In [3]:
CONFIG_PATH = "../configs/config.json"

## Data Pipeline

### Creating Custom Dataset Class

In [4]:
class CustomDataset(Dataset):
    """Dataset class For the Custom Dataset"""

    def __init__(self, csv_file: str = "../Data/DataSplits/test.csv", label_column: str = "Label"):
        """Initializer for the Dataset class.
        Args:
            csv_file (str): Path to the CSV file containing the dataset.
            label_column (str): The name of the column indicating the label.
        """
        try:
            self.data = pd.read_csv(csv_file)  # Assign a pandas data frame
        except FileNotFoundError:  # Raise an error if the file is not found
            raise FileNotFoundError(f"File not found: {csv_file}")

        # Define feature and label columns
        self.label_column = label_column
        # Omit the label column to create the list of feature columns
        self.feature_columns = self.data.columns.drop([self.label_column])

    def __getitem__(self, index: int) -> tuple[torch.Tensor, torch.Tensor]:
        """Returns a tuple (features, label) for the given index.
        Args:
            index (int): Index of the data sample to retrieve.
        Returns:
            tuple: (features, label) where features is a tensor of input features and label is the corresponding label.
        """
        # Use 'iloc' instead of 'loc' for efficiency
        features = self.data.iloc[index][self.feature_columns].values
        label = self.data.iloc[index][self.label_column]  # Extract the label for the given index
        return (torch.tensor(features, dtype=torch.float), torch.tensor(label, dtype=torch.long))

    def __len__(self) -> int:
        """Returns the amount of samples in the dataset."""
        return len(self.data)

### Utility Function

In [5]:
def clean_data(df: pd.DataFrame, logger: Logger) -> pd.DataFrame:
    """Cleans the input DataFrame.
    Args:
        df (pd.DataFrame): The input DataFrame to be cleaned.
        logger (Logger): Logger object for logging information.
    Returns:
        pd.DataFrame: The cleaned DataFrame.
    """
    show_dataframe_info = True  # Set to True to log DataFrame info

    # Log the initial state of the DataFrame
    if show_dataframe_info:
        logger.info(f"Initial DataFrame shape: {df.shape}")
        buffer = io.StringIO()  # Create a buffer to capture the info output
        df.info(buf=buffer)  # Store the output into the buffer
        logger.info(f"Initial DataFrame info:\n " + buffer.getvalue())

    # Example cleaning steps (customize as needed)
    df = df.drop_duplicates()  # Remove duplicates
    df = df.dropna()  # Drop rows with missing values

    if show_dataframe_info:
        # Reinitialize the buffer to clear any previous content in order to log the final dataframe info
        buffer = io.StringIO()
        df.info(buf=buffer)
        logger.info(f"Final DataFrame info:\n " + buffer.getvalue())

    return df

### Data Pipeline Function

In [6]:
def data_pipeline(
    logger: Logger,
    dataset_url: str,
    root_data_dir: str = "../Data",
    data_file_path: str = "Dataset.csv",
    data_splits_dir: str = "DataSplits",
    scaler_dir="Scalers",
    target_column: str = "Target",
    extra_dropped_columns: Optional[List[str]] = None,
    batch_size: int = 64,
    num_workers: int = 0,
    pin_memory: bool = False,
    drop_last: bool = True,
) -> tuple[
    Dataset, Dataset, Dataset, DataLoader, DataLoader, DataLoader, MinMaxScaler, MinMaxScaler
]:
    """This function prepares the train, test, and validation datasets.
    Args:
        logger (Logger): The logger instance to log messages.
        dataset_url (str): The URL to download the dataset from, if not found locally.
        root_data_dir (str): The root of the Data Directory
        data_file_path (str): The name of the original dataset (with .csv file extension).
        data_splits_dir (str): Path to the train, test, and validation datasets.
        scaler_dir (str): Path to the feature and label scalers.
        target_column (str): The name of the target column to predict.
        extra_dropped_columns (List[str], optional): Additional columns to drop from the features.
        batch_size (int): The dataloader's batch_size.
        num_workers (int): The dataloader's number of workers.
        pin_memory (bool): The dataloader's pin memory option.
        drop_last (bool): The dataloader's drop_last option.

    Returns:
        train_dataset (Dataset): Dataset Class for the training dataset.
        test_dataset (Dataset): Dataset Class for the test dataset.
        validation_dataset (Dataset): Dataset Class for the validation dataset.
        train_dataloader (DataLoader): The train dataloader.
        test_dataloader (DataLoader): The test dataloader.
        validation_dataloader (DataLoader): The validation dataloader.
        feature_scaler (MinMaxScaler): The scaler used to scale the features of the model input.
        label_scaler (MinMaxScaler): The scaler used to scale the labels of the model input.
    """
    if (
        not root_data_dir or not data_file_path or not data_splits_dir
    ):  # Check for empty strings at the beginning
        raise ValueError("File and directory paths cannot be empty strings.")
    DATA_ROOT = Path(root_data_dir)

    DATA_CLEAN_PATH = DATA_ROOT / data_file_path  # Set the path to the complete dataset

    if DATA_CLEAN_PATH.exists():
        logger.info(f"CSV file detected, reading from '{DATA_ROOT}'")
        df = pd.read_csv(
            DATA_CLEAN_PATH, dtype="float32"
        )  # Convert data to float32 instead of, float64
    else:
        logger.info(f"Downloading CSV file from '{dataset_url}'\nand saving into '{DATA_ROOT}'")
        try:
            os.makedirs(DATA_ROOT, exist_ok=True)  # Create the Data Root Directory
            # Download and read the data into a pandas dataframe
            df = pd.read_csv(
                dataset_url, dtype="float32"
            )  # Convert data to float32 instead of, float64

            # Clean the data before saving
            try:
                df = clean_data(df, logger)
            except Exception as e:
                raise RuntimeError(f"An unexpected error occurred cleaning the dataset:\n{e}")

            df.to_csv(DATA_CLEAN_PATH, index=False)  # Save the file, omitting saving the row index
        except OSError as e:
            raise RuntimeError(f"OS error occurred: {e}")
        except ParserError:
            raise RuntimeError(f"Failed to parse CSV from '{dataset_url}'")
        except ValueError as e:
            raise RuntimeError(f"Data cleaning error:\n{e}")
        except Exception as e:
            raise RuntimeError(
                f"An unexpected error occurred when downloading or saving the "
                f"dataset from '{dataset_url}' to '{DATA_CLEAN_PATH}':\n{e}"
            )

    # Define the paths for the data splits and scalers
    DATA_SPLITS_DIR = DATA_ROOT / data_splits_dir
    SCALER_DIR = DATA_ROOT / scaler_dir

    TRAIN_DATA_PATH = DATA_SPLITS_DIR / "train.csv"
    TEST_DATA_PATH = DATA_SPLITS_DIR / "test.csv"
    VALIDATION_DATA_PATH = DATA_SPLITS_DIR / "val.csv"

    FEATURE_SCALER_PATH = SCALER_DIR / "feature-scaler.joblib"
    LABEL_SCALER_PATH = SCALER_DIR / "label-scaler.joblib"

    # Dictate whether to use label scaler
    USE_LABEL_SCALER = False

    # Define the columns to drop from the features
    columns_to_drop = [target_column] + extra_dropped_columns

    # Define the Data Splits
    TRAIN_SPLIT_PERCENTAGE = 0.9
    VALIDATION_SPLIT_PERCENTAGE = 0.5

    if (
        os.path.exists(TRAIN_DATA_PATH)
        and os.path.exists(TEST_DATA_PATH)
        and os.path.exists(VALIDATION_DATA_PATH)
    ):
        logger.info(
            f"Train, Test, and Validation CSV datasets detected in '{DATA_SPLITS_DIR}.' Skipping generation and loading scaler(s)"
        )
        try:
            feature_scaler = joblib.load(FEATURE_SCALER_PATH)
            logger.info(f"Feature scaler stored in: ({FEATURE_SCALER_PATH})")
            if USE_LABEL_SCALER:
                joblib.dump(
                    label_scaler, LABEL_SCALER_PATH
                )  # Not used for this classification task
                logger.info(f"Label scaler stored in: ({LABEL_SCALER_PATH})")
            else:
                label_scaler = None  # Omit the label scaler loading

        except FileNotFoundError as e:
            raise RuntimeError(f"Scaler file not found: {e}")
        except EOFError as e:
            raise RuntimeError(f"Scaler file appears to be empty or corrupted: {e}")
        except Exception as e:
            raise RuntimeError(f"An unexpected error occurred when loading scalers: {e}")
    else:
        logger.info(
            f"Datasets not found in '{DATA_SPLITS_DIR}' or incomplete. Generating datasets..."
        )
        # os.makedirs(MODEL_ROOT, exist_ok=True)
        os.makedirs(DATA_SPLITS_DIR, exist_ok=True)  # Create the Data Splits Parent Directory
        os.makedirs(SCALER_DIR, exist_ok=True)  # Create the Scaler Parent Directory

        # Create the scaler objects
        feature_scaler = MinMaxScaler()
        if USE_LABEL_SCALER:
            label_scaler = MinMaxScaler()
        else:
            label_scaler = None  # Not used for this Classification task

        try:
            df_features = df.drop(columns=columns_to_drop, inplace=False)
            df_labels = df[
                [target_column]
            ]  # Instead of returning a pandas Series using "[]", return a dataframe using the "[[]]" to get a shape with (-1,1)
        except KeyError as e:
            raise KeyError(
                f"One or more specified columns to drop do not exist in the DataFrame: {e}"
            )

        # Split into smaller DataFrames for the Train, Test, and Validation splits
        X_train, X_inter, Y_train, Y_inter = train_test_split(
            df_features, df_labels, test_size=1 - TRAIN_SPLIT_PERCENTAGE, random_state=42
        )
        X_validation, X_test, Y_validation, Y_test = train_test_split(
            X_inter, Y_inter, test_size=1 - VALIDATION_SPLIT_PERCENTAGE, random_state=42
        )

        # Fit the scalers to the data
        feature_scaler.fit(X_train)
        # Only scale the labels if required
        if USE_LABEL_SCALER:
            label_scaler.fit(Y_train)  # Not used for this Classification task

        # Save the fitted scaler object
        try:
            joblib.dump(feature_scaler, FEATURE_SCALER_PATH)
            logger.info(f"Feature scaler stored in: ({FEATURE_SCALER_PATH})")
            # Save the Label Scaler if utilized
            if USE_LABEL_SCALER:
                joblib.dump(
                    label_scaler, LABEL_SCALER_PATH
                )  # Not used for this Classification task
                logger.info(f"Label scaler stored in: ({LABEL_SCALER_PATH})")
        except FileNotFoundError as e:
            raise RuntimeError(f"Save path not found: {e}")
        except Exception as e:
            raise RuntimeError(f"An unexpected error occurred when saving  Scaler(s): {e}")

        # Scale the rest of the data; returns numpy arrays
        X_train_scaled = feature_scaler.transform(X_train)
        X_validation_scaled = feature_scaler.transform(X_validation)
        X_test_scaled = feature_scaler.transform(X_test)

        if USE_LABEL_SCALER:  # Not used for this Classification task
            Y_train = label_scaler.transform(Y_train)
            Y_validation = label_scaler.transform(Y_validation)
            Y_test = label_scaler.transform(Y_test)

        logger.info(f"Train Features (Scaled) Shape: {X_train_scaled.shape}")
        logger.info(f"Validation Features (Scaled) Shape: {X_validation_scaled.shape}")
        logger.info(f"Test Features (Scaled) Shape: {X_test_scaled.shape}")

        if USE_LABEL_SCALER:
            logger.info(f"Train Labels (Scaled) Shape: {Y_train.shape}")
            logger.info(f"Validation Labels (Scaled) Shape: {Y_validation.shape}")
            logger.info(f"Test Labels (Scaled) Shape: {Y_test.shape}")
        else:
            logger.info(f"Train Labels Shape: {Y_train.shape}")
            logger.info(f"Validation Labels Shape: {Y_validation.shape}")
            logger.info(f"Test Labels Shape: {Y_test.shape}")

        # Define the column names of the features and label
        features_names = df_features.columns
        label_name = df_labels.columns

        # Create dataframes using the scaled data
        X_train_df = pd.DataFrame(X_train_scaled, columns=features_names)
        X_test_df = pd.DataFrame(X_test_scaled, columns=features_names)
        X_validation_df = pd.DataFrame(X_validation_scaled, columns=features_names)
        Y_train_df = pd.DataFrame(Y_train, columns=label_name)
        Y_test_df = pd.DataFrame(Y_test, columns=label_name)
        Y_validation_df = pd.DataFrame(Y_validation, columns=label_name)

        # Concatenate the features and labels back into a single DataFrame for each set
        train_data_frame = pd.concat([X_train_df, Y_train_df.reset_index(drop=True)], axis=1)
        test_data_frame = pd.concat([X_test_df, Y_test_df.reset_index(drop=True)], axis=1)
        validation_data_frame = pd.concat(
            [X_validation_df, Y_validation_df.reset_index(drop=True)], axis=1
        )

        # Saving the split data to csv files
        try:
            train_data_frame.to_csv(TRAIN_DATA_PATH, index=False)
            test_data_frame.to_csv(TEST_DATA_PATH, index=False)
            validation_data_frame.to_csv(VALIDATION_DATA_PATH, index=False)
        except FileNotFoundError as e:
            raise RuntimeError(f"Save path not found: {e}")
        except Exception as e:
            raise RuntimeError(
                f"An unexpected error occurred when saving datasets to CSV files:\n{e}"
            )

    # Creating Datasets from the stored datasets
    logger.info(f"INITIALIZING DATASETS")
    train_dataset = CustomDataset(csv_file=TRAIN_DATA_PATH, label_column=target_column)
    test_dataset = CustomDataset(csv_file=TEST_DATA_PATH, label_column=target_column)
    val_dataset = CustomDataset(csv_file=VALIDATION_DATA_PATH, label_column=target_column)

    logger.info(
        f"Creating DataLoaders with 'batch_size'=({batch_size}), 'num_workers'=({num_workers}), 'pin_memory'=({pin_memory}). Training dataset 'drop_last'=({drop_last})"
    )
    train_dataloader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=drop_last,
        shuffle=True,
    )
    validation_dataloader = DataLoader(
        dataset=val_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=drop_last,
        shuffle=False,
    )
    test_dataloader = DataLoader(
        dataset=test_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=drop_last,
        shuffle=False,
    )

    logger.info(
        f"Training DataLoader has ({len(train_dataloader)}) batches, Test DataLoader has ({len(test_dataloader)}) batches, Validation DataLoader has ({len(validation_dataloader)}) batches"
    )

    logger.info("==================================================================")
    for name, dataloader in [
        ("Train", train_dataloader),
        ("Validation", validation_dataloader),
        ("Test", test_dataloader),
    ]:
        features, labels = next(iter(dataloader))  # Get one batch

        logger.info(f"{name} Dataloader Batch Information")
        logger.info(f"Features Shape: '{features.shape}' |  DataTypes: '{features.dtype}'")
        logger.info(f"Labels Shape: '{labels.shape}'   |  DataTypes: '{labels.dtype}' ")
        logger.info("==================================================================")

    return (
        train_dataset,
        test_dataset,
        val_dataset,
        train_dataloader,
        test_dataloader,
        validation_dataloader,
        feature_scaler,
        label_scaler,
    )

## Agent Architecture

### Module Layer

In [7]:
class ModuleLayer(torch.nn.Module):
    """Class for the individual layer blocks."""

    def __init__(self, intermediate_dim=32, dropout_rate=0.1):
        """Initializer for the 'ModuleLayer' class.
        Args:
            intermediate_dim (int): The dimension of the intermediate layer.
            dropout_rate (float): The dropout rate to apply after the ReLU activation.
        """
        super().__init__()
        self.mod_linear = torch.nn.Linear(intermediate_dim, intermediate_dim)
        self.mod_norm = torch.nn.LayerNorm(normalized_shape=intermediate_dim)
        self.mod_relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p=dropout_rate)

    def forward(self, x):
        """Forward pass of the layer block.
        Args:
            x (torch.Tensor): Input tensor.
        Returns:
            torch.Tensor: Output tensor passing the input through the layer operations.
        """
        residual = x
        x = self.mod_linear(x)
        x = self.mod_norm(x)
        x = self.mod_relu(x)
        x = self.dropout(x)
        x += residual
        return x

### Agent Class

In [8]:
class Agent(torch.nn.Module):
    """Class for Agent Structure using multiple Layer Blocks."""

    def __init__(self, cfg):
        """Initializer for the 'Agent' class.
        Args:
            cfg (dict): Configuration dictionary containing model parameters.
        """
        super().__init__()
        self.linear = torch.nn.Linear(
            in_features=cfg["in_dim"], out_features=cfg["intermediate_dim"]
        )

        self.layers = torch.nn.Sequential(
            *[
                ModuleLayer(
                    intermediate_dim=cfg["intermediate_dim"], dropout_rate=cfg["dropout_rate"]
                )
                for _ in range(int(cfg["num_blocks"]))
            ]
        )

        self.out = torch.nn.Linear(in_features=cfg["intermediate_dim"], out_features=cfg["out_dim"])

    def forward(self, x):
        """Forward pass through the Agent's Layers.
        Args:
            x (torch.Tensor): Input tensor.
        Returns:
            x (torch.Tensor): Output tensor after passing through the network.
        """
        x = self.linear(x)
        x = self.layers(x)
        x = self.out(x)
        return x

## Training

### Log Iteration Functions

In [9]:
def log_iteration(logger: Logger, batch_idx: int, total_batches: int, loss_value: float) -> None:
    """Logs the loss of the current batch."""
    logger.info(f"Epoch batch [{batch_idx}/{total_batches}] | Loss: {loss_value:.7f}")

In [10]:
def log_epoch_iteration(logger: Logger, epoch: int, avg_epoch_loss: float) -> None:
    """Log Current Metrics accumulated in the current epoch iteration.
    Args:
        logger (Logger): The logger instance to log messages.
        epoch (int): the current iteration.
        avg_epoch_loss (float): The average loss of the current epoch.
    """
    if avg_epoch_loss:
        logger.info(f"=====================  [EPOCH ({epoch}) LOGGING]  =====================")
        logger.info("| AVERAGES of THIS EPOCH:")
        logger.info(f"| ACCUMULATED LOSS: {avg_epoch_loss:.7f}")
        logger.info(f"===========================================================")

    else:
        logger.warning("No Data collected for this epoch to log")

### Evaluate Model Function

In [11]:
def evaluate_model(
    logger: Logger,
    model: Module,
    dataloader: DataLoader,
    current_epoch: int = None,
    max_epochs: int = None,
    device: str = "cpu",
) -> float:
    """
    Evaluates the model on a given dataset and returns the average loss.
    Args:
        logger (Logger): The logger instance to log messages.
        model (Module): The Model.
        dataloader (DataLoader): The dataloader to calculate average loss with.
        current_epoch (int): The current epoch [optional].
        max_epochs (int): The maximum number of epochs [optional].
        device (str): The device that the calculations will take place on.
    Returns:
        avg_loss (float): The calculated average loss.
    """
    model.eval()
    total_loss = 0.0
    # loss_fn = torch.nn.MELoss(reduction='sum') # Use reduction='sum' instead of 'mean' for total loss
    loss_fn = torch.nn.L1Loss(
        reduction="sum"
    )  # Use Mean Absolute Loss with reduction='sum' instead of 'mean' for total loss
    if len(dataloader.dataset) == 0:
        logger.warning("Warning: Evaluation dataset is empty. Skipping evaluation.")
        return float("nan")

    with torch.no_grad():
        for batch_inputs, batch_labels in dataloader:
            batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.unsqueeze(dim=-1).to(
                device
            )
            outputs = model(batch_inputs)
            loss = loss_fn(outputs, batch_labels)
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader.dataset)  # Calculate the average loss on the dataset

    if current_epoch and max_epochs:  # If the function was called in the training loop
        logger.info(
            f"===================  [Epoch ({current_epoch}/{max_epochs})]  ==================="
        )
        logger.info(f"Entire Validation Dataset Average Loss: {avg_loss:.4f}")
        logger.info(f"====================================================")

    else:  # If the function was called outside of the training loop
        logger.info(f"===============================================")
        logger.info(f"Entire Dataset Average Loss: {avg_loss:.4f} ")
        logger.info(f"=====================================================")

    return avg_loss

### Close the logger object and exit. Used during Exception Handling.

In [12]:
def close_log_with_exit(exit_code: int) -> None:
    """Exits the application with the given exit code.
    Args:
        exit_code (int): The exit code to return when exiting the application.
    """
    # This line closes all handlers and releases the file lock
    logging.shutdown()
    exit(exit_code)

## Miscelaneous Utility Functions

### Checks for current running mode

In [13]:
def is_notebook():
    """Checks if the code is running in a Jupyter notebook environment.
    Returns:
        bool: True if running in a Jupyter notebook, False otherwise.
    """
    try:
        shell = get_ipython().__class__.__name__

        # print(f"Detected shell: {shell}", flush=True)

        if shell == "ZMQInteractiveShell":
            return True  # Jupyter notebook or qtconsole
        elif shell == "TerminalInteractiveShell":
            return False  # Terminal running IPython
        else:
            return False  # Other types
    except NameError:
        return False

### Creating Logger Function

In [14]:
def setup_logger(config: dict, propogate: bool = False) -> Logger:
    """Sets up and returns a named logger based on the provided config dictionary. The new logger will have different handlers based on the config.

    Args:
        config (dict): Dictionary containing logging configuration.
        propogate (bool): Whether to allow log messages to propagate to ancestor loggers.
    Returns:
        Logger: Configured logger instance.
    """

    logger_name = config.get("logger_name", "main")
    log_to_file = config.get("log_to_file", True)  # Set whether to log to a logfile or not
    log_file = config.get("log_file", "logs/app.log")  # Get the log file path
    log_lvl = config.get("log_level", "INFO")
    log_level = getattr(logging, log_lvl.upper(), logging.INFO)  # Set fallback if invalid input
    log_mode = config.get("log_mode", "w")  # Set the log file mode
    log_format = config.get("log_format", "%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    date_format = config.get("date_format", "%Y-%m-%d %H:%M:%S")
    log_to_console = config.get("log_to_console", True)  # Set whether to log to console or not

    handlers = []  # Initialize the list of logging handlers

    logger = logging.getLogger(logger_name)  # Create logger object with the specified name

    if not log_to_file and not log_to_console:
        # If no handlers are specified by the config
        print(
            f"Warning: No logging handlers configured for {logger_name}.\nVerbose Logging will be disabled.\nIn 'config/config.json', set ['log_to_file': true] or ['log_to_console': true] if you want to change the logging behavior.",
            flush=True,
        )
    else:
        # Create log parent directory if it doesn't exist
        parent_dir = os.path.dirname(log_file)  # Get the parent directory of the log file
        if parent_dir and parent_dir != ".":
            try:
                os.makedirs(name=parent_dir, exist_ok=True)
                print(
                    f"Parent directory '{parent_dir}' used to store the log file.", flush=True
                )  # flush=True to ensure the message is printed immediately
            except OSError as e:
                print(
                    f"Error creating directory '{parent_dir}': {e} INFO: Using default log file 'app.log' instead.",
                    flush=True,
                )
                log_file = "app.log"  # Fall back to a default log file if problem occurs.

        # Remove all old handlers inherrited from the root logger
        for handler in logger.handlers[:]:
            handler.close()
            logger.removeHandler(handler)

        formatter = logging.Formatter(
            fmt=log_format, datefmt=date_format
        )  # Create a formatter for the log messages

        if log_to_console:
            console_handler = (
                logging.StreamHandler()
            )  # Initialize sending log messages to the console (stdout)
            console_handler.setFormatter(formatter)  # Set the formatter for the console handler
            handlers.append(console_handler)  # Add the console_handler to the list of handlers
        if log_to_file:
            file_handler = logging.FileHandler(
                filename=log_file, mode=log_mode
            )  # Initialize sending log messages to a file
            file_handler.setFormatter(formatter)  # Set the style for the console handler
            handlers.append(file_handler)  # Add the file_handler to the list of handlers

        # Add the handlers to the logger
        for handler in handlers:
            logger.addHandler(handler)

    logger.setLevel(log_level)  # Set logger minimum log level

    logger.propagate = propogate  # Prevent the log messages from being propagated to the root logger; gets rid of the root logger's default handlers,

    return logger

### Retrieve Logger

In [15]:
def retrieve_logger(name: str = "root") -> Logger:
    """
    Retrieves a named logger. If no handlers are attached, returns a root logger instance.
    Args:
        name (str): The name of the logger to retrieve.
    Returns:
        Logger: The logger instance.
    """
    logger = logging.getLogger(name)

    if not logger.handlers:
        print(f"Retrieving root logger.", flush=True)
        return logging.getLogger()

    return logger

### Close file handlers and exit

In [16]:
def close_and_exit(logger: Logger, exit_code: int) -> None:
    """Closes all handlers of the logger and exits the program with the given exit code.
    Args:
        logger (Logger): The logger instance to close.
        exit_code (int): The exit code to terminate the program with.
    """
    print("Note: Closing any named loggers...", flush=True)
    used_handlers = logger.handlers[:]
    for handler in used_handlers:
        handler.close()
        logger.removeHandler(handler)

    print("Note: Closing any root logger...", flush=True)
    for handler in logging.root.handlers[:]:
        handler.close()
        logging.root.removeHandler(handler)

    print(f"Exiting program with exit code {exit_code}.", flush=True)

    if is_notebook():
        print(
            "Detected Jupyter Notebook environment. Skipping sys.exit() to avoid kernel interruption.",
            flush=True,
        )
    else:
        sys.exit(exit_code)

In [ ]:
# #  --- Load Config File ---
# try:
#     with open(file=CONFIG_PATH, mode="r") as f:
#         json_args = json.load(f)
# except FileNotFoundError:
#     print(
#         f"Config file not found. Please ensure '{CONFIG_PATH}' exists. Modify 'CONFIG_PATH' in Global Variables section if needed.",
#         flush=True,
#     )
#     sys.exit(1)
# except Exception as e:
#     print(f"Unexpected error loading config file: {e}. Exiting.")
#     sys.exit(1)

## Main Loop

In [20]:
def main(global_config) -> int:
    """Main function to run model sampling.
    Args:
                global_config: The arguments from the config file.
    Returns:
        int: Exit code (0 for success, non-zero for failure).
    """
    # Create logger object
    logger = retrieve_logger(global_config["logging"]["logger_name"])
    logger.info("STARTING MAIN FUNCTION")

    DEVICE = "cpu"

    BASE_CONFIG = global_config["model"]
    TRAINING_CONFIG = global_config["training"]
    PIPELINE_CONFIG = global_config["data"]
    parser_args = global_config["parser_defaults"]

    # --- Data Preparation Pipeline ---
    logger.info("RUNNING THE DATA PIPELINE")
    try:
        # Use dictionary unpacking to pass the PIPELINE_CONFIG parameters to the data_pipeline function
        (
            train_dataset,
            test_dataset,
            validation_dataset,
            train_dataloader,
            test_dataloader,
            validation_dataloader,
            feature_scaler,
            label_scaler,
        ) = data_pipeline(
            logger=logger,
            **PIPELINE_CONFIG,
            batch_size=parser_args["dataloader_batch_size"],
            num_workers=parser_args["dataloader_num_workers"],
            pin_memory=parser_args["dataloader_pin_memory"],
            drop_last=True,
        )
    except ValueError as e:
        logger.error(f"Caught a 'value' error: {e}", exc_info=True, stack_info=True)
        return 1
    except RuntimeError as e:
        logger.error(f"Caught a 'runtime' error: {e}", exc_info=True, stack_info=True)
        return 1
    except MemoryError as e:
        logger.error(
            f"Memory Error: {e}. Consider reducing the DataLoader's batch size or model complexity.",
            exc_info=True,
            stack_info=True,
        )
        return 1
    except KeyboardInterrupt:
        logger.error(
            "Training interrupted by user (KeyboardInterrupt).", exc_info=True, stack_info=True
        )
        return 1
    except Exception as e:
        logger.error(
            f"An unexpected error occurred during model training: {e}",
            exc_info=True,
            stack_info=True,
        )
        return 1

    return 0

## Main Process

In [21]:
# Call this function, during script execution; Main script entry point
if __name__ == "__main__":
    # --- Begin Timing Main Script Execution Time ---
    main_start_time = time.time()

    #  --- Load Config File ---
    try:
        with open(file=CONFIG_PATH, mode="r") as f:
            json_args = json.load(f)
    except FileNotFoundError:
        print(
            f"Config file not found. Please ensure '{CONFIG_PATH}' exists. Modify 'CONFIG_PATH' in Global Variables section if needed.",
            flush=True,
        )
        sys.exit(1)
    except Exception as e:
        print(f"Unexpected error loading config file: {e}. Exiting.")
        sys.exit(1)

    # --- Logging Initialization Section ---
    log_to_console = json_args["logging"]["log_to_console"]  # Set whether to log to console or not
    log_to_file = json_args["logging"]["log_to_file"]  # Set whether to log to a logfile or not
    logger_config = json_args["logging"]

    # Configure the root logger for any backup logging
    logging.basicConfig(
        level=logging.CRITICAL
    )  # Set root logger to highest level to suppress unwanted logs

    # Create the named logger only if the user wants to log to console or file
    if log_to_console or log_to_file:
        logger = setup_logger(config=logger_config, propogate=False)
    # Check if the user disabled both logging methods and resort to the root logger with no handlers
    elif not log_to_file and not log_to_console:
        print(
            f"========================================\nWarning: No logging handlers configured for logger, using root logger.\nVerbose Logging will be disabled.\nTraining progress will be shown using tqdm.\nIf you want to change the logging behavior:\nIn 'config/config.json', set ['log_to_file': true] or ['log_to_console': true]\n========================================\n ",
            flush=True,
        )
        logger = logging.getLogger()  # Use the root logger

        # Remove all handlers inherrited from the root logger, if any exist
        for handler in logger.handlers[:]:
            handler.close()
            logger.removeHandler(handler)

    ## --- Call Main Script Section ---
    logger.info("CALLING MAIN SCRIPT...", exc_info=False, stack_info=False)

    # Call the main function with both the parser arguments and the 'config.json' file
    ret = main(json_args)

    # --- Determine final message based on return code ---
    if ret == 0:
        if not log_to_console and not log_to_file:
            print("FINISHED MAIN SCRIPT.\n", flush=True)
        else:
            logger.info("TERMINATING PROGRAM")
    else:
        if not log_to_console and not log_to_file:
            print("MAIN SCIPT ERROR", flush=True)
            logger.error("MAIN SCIPT ERROR")

    # --- Closes any logger file handlers and exits the program ---
    close_and_exit(logger, ret)

Parent directory 'logs' used to store the log file.


2025-10-24 13:09:03 - main - INFO - CALLING MAIN SCRIPT...
2025-10-24 13:09:03 - main - INFO - STARTING MAIN FUNCTION
2025-10-24 13:09:03 - main - INFO - RUNNING THE DATA PIPELINE
2025-10-24 13:09:03 - main - INFO - CSV file detected, reading from 'Data'
2025-10-24 13:09:03 - main - INFO - Train, Test, and Validation CSV datasets detected in 'Data\DataSplits.' Skipping generation and loading scaler(s)
2025-10-24 13:09:03 - main - INFO - Feature scaler stored in: (Data\Scalers\feature-scaler.joblib)
2025-10-24 13:09:03 - main - INFO - INITIALIZING DATASETS
2025-10-24 13:09:04 - main - INFO - Creating DataLoaders with 'batch_size'=(64), 'num_workers'=(0), 'pin_memory'=(False). Training dataset 'drop_last'=(True)
2025-10-24 13:09:04 - main - INFO - Training DataLoader has (14745) batches, Test DataLoader has (819) batches, Validation DataLoader has (819) batches
2025-10-24 13:09:04 - main - INFO - ==================================================================
2025-10-24 13:09:04 - mai

Note: Closing any named loggers...
Note: Closing any root logger...
Exiting program with exit code 0.
Detected Jupyter Notebook environment. Skipping sys.exit() to avoid kernel interruption.


In [ ]:
def get_prediction(model, features):
    """Get a prediction from the model for given features.
    Args:
        model (Agent): The Model.
        features (torch.tensor): The input features for prediction.
    Returns:
        prediction (float): The predicted value."""
    model.eval()
    with torch.no_grad():
        if not isinstance(features, torch.Tensor):
            features = torch.tensor(features, dtype=torch.float)
        prediction = model(features.unsqueeze(0))  # Add batch dimension
    return prediction.item()

In [22]:
#  --- Load Config File ---
try:
    with open(file=CONFIG_PATH, mode="r") as f:
        global_config = json.load(f)
except FileNotFoundError:
    print(
        f"Config file not found. Please ensure '{CONFIG_PATH}' exists. Modify 'CONFIG_PATH' in Global Variables section if needed.",
        flush=True,
    )
    sys.exit(1)
except Exception as e:
    print(f"Unexpected error loading config file: {e}. Exiting.")
    sys.exit(1)

In [31]:
logger = retrieve_logger(global_config["logging"]["logger_name"])

Retrieving root logger.


In [23]:
DEVICE = "cpu"

BASE_CONFIG = global_config["model"]
TRAINING_CONFIG = global_config["training"]
PIPELINE_CONFIG = global_config["data"]
parser_args = global_config["parser_defaults"]

In [24]:
trained_policy = Agent(BASE_CONFIG)

In [25]:
SAVE_LOCATION = "../models/my-model.pt"

In [26]:
params = torch.load(f=SAVE_LOCATION, weights_only=True)

In [27]:
trained_policy.load_state_dict(params)

<All keys matched successfully>

In [29]:
trained_policy.eval()

Agent(
  (linear): Linear(in_features=9, out_features=128, bias=True)
  (layers): Sequential(
    (0): ModuleLayer(
      (mod_linear): Linear(in_features=128, out_features=128, bias=True)
      (mod_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (mod_relu): ReLU()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): ModuleLayer(
      (mod_linear): Linear(in_features=128, out_features=128, bias=True)
      (mod_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (mod_relu): ReLU()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (2): ModuleLayer(
      (mod_linear): Linear(in_features=128, out_features=128, bias=True)
      (mod_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (mod_relu): ReLU()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (3): ModuleLayer(
      (mod_linear): Linear(in_features=128, out_features=128, bias=True)
      (mod_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    

In [ ]:
try:
    # Use dictionary unpacking to pass the PIPELINE_CONFIG parameters to the data_pipeline function
    (
        train_dataset,
        test_dataset,
        validation_dataset,
        train_dataloader,
        test_dataloader,
        validation_dataloader,
        feature_scaler,
        label_scaler,
    ) = data_pipeline(
        logger=logger,
        **PIPELINE_CONFIG,
        batch_size=parser_args["dataloader_batch_size"],
        num_workers=parser_args["dataloader_num_workers"],
        pin_memory=parser_args["dataloader_pin_memory"],
        drop_last=True,
    )
except ValueError as e:
    logger.error(f"Caught a 'value' error: {e}", exc_info=True, stack_info=True)
except RuntimeError as e:
    logger.error(f"Caught a 'runtime' error: {e}", exc_info=True, stack_info=True)
except MemoryError as e:
    logger.error(
        f"Memory Error: {e}. Consider reducing the DataLoader's batch size or model complexity.",
        exc_info=True,
        stack_info=True,
    )
except KeyboardInterrupt:
    logger.error(
        "Training interrupted by user (KeyboardInterrupt).", exc_info=True, stack_info=True
    )
except Exception as e:
    logger.error(
        f"An unexpected error occurred during model training: {e}",
        exc_info=True,
        stack_info=True,
    )

In [32]:
# --- Testing Trained Model ---
print("\nTESTING THE TRAINED POLICY:")
test_loss = evaluate_model(
    logger=logger,
    model=trained_policy,
    dataloader=test_dataloader,
    current_epoch=None,
    max_epochs=None,
    device="cpu",
)


TESTING THE TRAINED POLICY:


c:\Users\brianperez\AppData\Local\anaconda3\envs\CC_env\Lib\site-packages\torch\nn\modules\loss.py:128: UserWarning: Using a target size (torch.Size([64, 1])) that is different to the input size (torch.Size([64, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


In [33]:
print(test_loss)

7.39273344572988


In [37]:
def sample_model(model, dataset, label_scaler, feature_scaler=None, samples=5) -> None:
    """Function to sample a model on a given dataset and provide a detailed ouput of the predictions, actual values, and loss for each sample.
    Args:
        model (Agent): The Model.
        dataset (Dataset): The dataset to sample from
        samples (int): The number of samples to take from the dataset. If 0 or less, defaults to 5. If more than dataset size, adjusts to dataset size.
    """
    loss_fn = torch.nn.CrossEntropyLoss(
        reduction="sum"
    )  # Declare the los function to be the Mean Absolute Error loss

    if samples <= 0:
        print("Number of samples must be greater than zero. Setting samples to 5")
        samples = 5

    if samples > len(dataset):
        print(
            f"Requested number of samples ({samples}) exceeds dataset size ({len(dataset)}). Adjusting to dataset size."
        )
        samples = len(dataset)

    total_loss = 0.0
    with torch.no_grad():
        for index in range(samples):
            (features, label) = dataset[index]
            # print(features[:5])   # Print the first 5 features for inspection
            pred = model(features)  # Get the models prediction baesd on the input features
            loss = loss_fn(
                pred, label
            )  # Calculate the loss, adding an extra dimension to the label to match the prediction shape
            total_loss += loss.item()

            # Un-normalize prediction and label using the inverse function and get the scalar value from the numpy arrays
            pred_unscaled = label_scaler.inverse_transform(pred.cpu().numpy().reshape(-1, 1))[0, 0]

            label_unscaled = label_scaler.inverse_transform(label.cpu().numpy().reshape(-1, 1))[
                0, 0
            ]

            # Un-normalize features if scaler is provided
            if feature_scaler is not None:
                features_unscaled = feature_scaler.inverse_transform(
                    features.cpu().numpy().reshape(1, -1)
                )[0]
                print(f"Un-normalized features: {features_unscaled}")
            else:
                print(f"Normalized features: {features.numpy()}")

            print(
                f"Model predicts: {pred.item():.7f} | Actual: {label:.7f} | Loss: {loss.item():.7f}"
            )
            print(
                f"Un-normalized prediction: {pred_unscaled:.4f} | Un-normalized actual: {label_unscaled:.4f}\n"
            )
    avg_loss = total_loss / samples
    print(f"Average loss over the samples: {avg_loss:.7f}")

In [38]:
sample_model(trained_policy, test_dataset, label_scaler, feature_scaler, samples=2)

AttributeError: 'NoneType' object has no attribute 'inverse_transform'